Install needed libraries

In [1]:
%pip install requests
%pip install PyGithub
%pip install openai
%pip install --upgrade openai
%pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 24.5 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


Imports

In [2]:
from openai import AzureOpenAI
import json
import requests
import re
from dotenv import load_dotenv
import os

Function to check if input is already a GitHub repo

In [3]:
def is_github_repo_link(env):
    return env.startswith("https://github.com/") or env.startswith("http://github.com/")

Prompt for user

In [16]:
prompt = input("Enter a name of the environment or link to GitHub repo: ")
flag = True
if is_github_repo_link(prompt) == False:
   flag = False

Enter a name of the environment or link to GitHub repo: https://github.com/nodejs/node.git


OpenAI API for getting repositories

In [17]:
load_dotenv()
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)
def generate_github_repos(prompt, model="gpt-4o"):
    system_prompt = "You are an AI assistant that suggests relevant GitHub repositories based on a project description."
    user_prompt = f"Suggest GitHub repositories for a project that involves the following technologies: {prompt}. Provide only the URLs of the repositories."

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    suggested_repos = response.choices[0].message.content.strip()
    repo_urls = re.findall(r'(https?://github\.com/[^\s]+)', suggested_repos)
    return repo_urls
repo_urls = generate_github_repos(prompt)
if repo_urls:
    print(repo_urls)
else:
    print("Failed to fetch repository URLs.")

['https://github.com/expressjs/express', 'https://github.com/koajs/koa', 'https://github.com/hapijs/hapi', 'https://github.com/nestjs/nest', 'https://github.com/SocketCluster/socketcluster', 'https://github.com/socketio/socket.io', 'https://github.com/mongodb/node-mongodb-native', 'https://github.com/sequelize/sequelize', 'https://github.com/redis/node-redis', 'https://github.com/jaredhanson/passport', 'https://github.com/remy/nodemon', 'https://github.com/vuejs/vue-next', 'https://github.com/facebook/react', 'https://github.com/angular/angular', 'https://github.com/webpack/webpack', 'https://github.com/jest-community/jest', 'https://github.com/mochajs/mocha']


Function for searching GitHub repositories

In [18]:
def retrieve_files(repo_urls):
    file_names = ['README.md', 'install.md', 'package.json']
    files_content = {file_name: [] for file_name in file_names}

    for url in repo_urls:
        if url.endswith('.git'):
            repo_name = url[:-4].split('github.com/')[-1]
        else:
            repo_name = url.split('github.com/')[-1]

        branches = ['main', 'master']
        for branch in branches:
            api_url = f"https://api.github.com/repos/{repo_name}/git/trees/{branch}?recursive=1"

            try:
                response = requests.get(api_url)
                if response.status_code == 200:
                    tree = response.json().get('tree', [])
                    for file_info in tree:
                        file_path = file_info['path']
                        file_name = file_path.split('/')[-1]
                        if file_name in file_names:
                            file_url = f"https://raw.githubusercontent.com/{repo_name}/{branch}/{file_path}"
                            file_response = requests.get(file_url)
                            if file_response.status_code == 200:
                                files_content[file_name].append(file_response.text)
                                print(f"Successfully retrieved {file_name} from {file_url}")
                            else:
                                print(f"Failed to retrieve {file_name} from {file_url}. Status code: {file_response.status_code}")
                    break
                elif response.status_code == 404:
                    print(f"Branch '{branch}' not found for repository {repo_name}.")
                else:
                    print(f"Failed to retrieve file list from {api_url}. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error while trying to get file list from {api_url}: {e}")

    return files_content

Functions for filtering repositories

In [19]:
def fetch_repo_details(repo_url):
    repo_api_url = repo_url.replace("https://github.com", "https://api.github.com/repos")
    response = requests.get(repo_api_url)
    if response.status_code == 200:
        return response.json()
    return None

def get_top_repos(repo_urls, top_n=3):
    repo_details = []
    for repo_url in repo_urls:
        details = fetch_repo_details(repo_url)
        if details:
            repo_details.append(details)
    sorted_repos = sorted(repo_details, key=lambda x: x['stargazers_count'], reverse=True)
    top_repos = sorted_repos[:top_n]
    return [repo['html_url'] for repo in top_repos]

top_repos = get_top_repos(repo_urls)
top_repos

['https://github.com/facebook/react',
 'https://github.com/angular/angular',
 'https://github.com/nestjs/nest']

In [20]:
files_content = retrieve_files(top_repos)

Successfully retrieved README.md from https://raw.githubusercontent.com/facebook/react/main/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/facebook/react/main/compiler/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/facebook/react/main/compiler/apps/playground/README.md
Successfully retrieved package.json from https://raw.githubusercontent.com/facebook/react/main/compiler/apps/playground/package.json
Successfully retrieved README.md from https://raw.githubusercontent.com/facebook/react/main/compiler/crates/react_build_hir/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/facebook/react/main/compiler/crates/react_diagnostics/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/facebook/react/main/compiler/crates/react_estree/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/facebook/react/main/compiler/crates/react_estree_codeg

Analyze and pre-process the data

In [21]:
def parse_readme(content):
    sections = {}
    current_section = None
    lines = content.split('\n')

    for line in lines:
        header_match = re.match(r'^(#+)\s+(.*)', line)
        if header_match:
            section_name = header_match.group(2).strip()
            current_section = section_name
            sections[current_section] = []
            print(f"New section: {section_name}")
        elif current_section:
            sections[current_section].append(line)

    for section in sections:
        sections[section] = "\n".join(sections[section])

    print("Parsed sections:", sections)
    return sections

In [22]:
def parse_package_json(content):
    try:
        if not content.strip():
            print("Empty content for package.json.")
            return {}

        package_data = json.loads(content)
        return {
            "dependencies": package_data.get("dependencies", {}),
            "devDependencies": package_data.get("devDependencies", {}),
            "scripts": package_data.get("scripts", {})
        }
    except json.JSONDecodeError as e:
        print(f"Error parsing package.json: {e}")
        return {}

In [23]:
def collect_context(files_content):
    context = {
        'README.md': [],
        'install.md': [],
        'package.json': []
    }

    for file_name, contents in files_content.items():
        if file_name == 'README.md':
            for content in contents:
                context[file_name].append(parse_readme(content))
        elif file_name == 'package.json':
            for content in contents:
                if content.strip():
                    parsed_content = parse_package_json(content)
                    if parsed_content:
                        context[file_name].append(parsed_content)
        else:
            context[file_name].extend(contents)

    return context
context=collect_context(files_content)
import pprint
pprint.pprint(context)

Izlaz streaminga skraćen je na ovoliko posljednjih redaka: 5000.
                                '3. Create sqlite database and create tables: '
                                '`npx prisma db push`\n'
                                '4. Start server: `npm run start:dev`\n',
                'Prisma GraphQL schema first sample': '\n'
                                                      'This sample project '
                                                      'uses sqlite as the '
                                                      'relational database. To '
                                                      'use a different '
                                                      'database, check the '
                                                      '[Prisma '
                                                      'docs](https://www.prisma.io/docs/getting-started).\n'},
               {'Description': '\n'
                               '[Nest](https://github.com/nestjs/nest

In [24]:
def extract_relevant_info(context):
    relevant_info = {
        "dependencies": [],
        "devDependencies": [],
        "scripts": [],
        "readme_sections": {}
    }

    for pkg in context.get('package.json', []):
        if "dependencies" in pkg:
            relevant_info["dependencies"].extend(pkg["dependencies"].keys())
        if "devDependencies" in pkg:
            relevant_info["devDependencies"].extend(pkg["devDependencies"].keys())

    for readme in context.get('README.md', []):
        for section, content in readme.items():
            if "setup" in section.lower() or "installation" in section.lower():
                relevant_info["readme_sections"][section] = content

    return relevant_info

relevant_info = extract_relevant_info(context)
pprint.pprint(relevant_info)

{'dependencies': ['@babel/core',
                  '@babel/parser',
                  '@babel/plugin-syntax-typescript',
                  '@babel/plugin-transform-block-scoping',
                  '@babel/plugin-transform-modules-commonjs',
                  '@babel/preset-react',
                  '@babel/preset-typescript',
                  '@babel/traverse',
                  '@babel/types',
                  '@heroicons/react',
                  '@monaco-editor/react',
                  '@playwright/test',
                  '@use-gesture/react',
                  'fs',
                  'hermes-eslint',
                  'hermes-parser',
                  'invariant',
                  'lz-string',
                  'monaco-editor',
                  'next',
                  'notistack',
                  'prettier',
                  'pretty-format',
                  're-resizable',
                  'react',
                  'react-compiler-runtime',
                  'react

In [25]:
def format_input_for_model(relevant_info):
    input_data = {
        "prompt": "Based on the following information, generate a devcontainer.json file:\n\n"
    }

    input_data["prompt"] += "Dependencies:\n"
    input_data["prompt"] += ", ".join(relevant_info["dependencies"]) + "\n\n"

    input_data["prompt"] += "DevDependencies:\n"
    input_data["prompt"] += ", ".join(relevant_info["devDependencies"]) + "\n\n"

    input_data["prompt"] += "Scripts:\n"
    input_data["prompt"] += json.dumps(relevant_info["scripts"], indent=2) + "\n\n"

    input_data["prompt"] += "README Sections:\n"
    for section, content in relevant_info["readme_sections"].items():
        input_data["prompt"] += f"{section}:\n{content}\n\n"

    return input_data

input_data = format_input_for_model(relevant_info)
pprint.pprint(input_data)

{'prompt': 'Based on the following information, generate a devcontainer.json '
           'file:\n'
           '\n'
           'Dependencies:\n'
           '@babel/core, @babel/parser, @babel/plugin-syntax-typescript, '
           '@babel/plugin-transform-block-scoping, '
           '@babel/plugin-transform-modules-commonjs, @babel/preset-react, '
           '@babel/preset-typescript, @babel/traverse, @babel/types, '
           '@heroicons/react, @monaco-editor/react, @playwright/test, '
           '@use-gesture/react, fs, hermes-eslint, hermes-parser, invariant, '
           'lz-string, monaco-editor, next, notistack, prettier, '
           'pretty-format, re-resizable, react, react-compiler-runtime, '
           'react-dom, @babel/generator, @babel/types, chalk, invariant, '
           'pretty-format, zod, zod-validation-error, @babel/core, '
           '@babel/parser, @babel/plugin-proposal-private-methods, '
           'hermes-parser, zod, zod-validation-error, invariant, '
       

Generate JSON file for Dev Container with OpenAI API

In [26]:
load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

def generate_dev_container_json(context, model="gpt-4o"):
    system_prompt = "You are an AI assistant that creates JSON file for dev container.\n\n"
    user_prompt = f"Based on the following information, generate a devcontainer.json file:\n\n {context}"

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )


        if response.choices:
            for choice in response.choices:
                if choice.finish_reason == 'stop' and choice.message:
                    generated_json = choice.message.content.strip()
                    try:

                        start_index = generated_json.find('{')
                        end_index = generated_json.rfind('}') + 1
                        json_str = generated_json[start_index:end_index]
                        parsed_json = json.loads(json_str)

                        print("Generated Dev Container JSON Configuration:")
                        print(json.dumps(parsed_json, indent=4))
                        return parsed_json
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON: {e}")
                        return None
                else:
                    print(f"Invalid choice received: {choice}")
                    return None
        else:
            print("No valid choices found in API response.")
            return None

    except Exception as ex:
        print(f"Error generating devcontainer.json: {ex}")
        return None

generated_devcontainer = generate_dev_container_json(input_data)

Generated Dev Container JSON Configuration:
{
    "name": "Development Container",
    "dockerFile": "Dockerfile",
    "settings": {
        "terminal.integrated.shell.linux": "/bin/bash",
        "editor.formatOnSave": true,
        "extensions": [
            "dbaeumer.vscode-eslint",
            "esbenp.prettier-vscode",
            "ms-vscode.vscode-typescript-next"
        ]
    },
    "postCreateCommand": "npm install",
    "containerUser": "node",
    "forwardPorts": [
        3000
    ],
    "portsAttributes": {
        "3000": {
            "label": "Application",
            "onAutoForward": "notify"
        }
    },
    "mounts": [
        "source=${localWorkspaceFolder}/node_modules,target=/workspace/node_modules,type=bind"
    ],
    "workspaceMount": "source=${localWorkspaceFolder},target=workspace,type=bind,consistency=cached",
    "workspaceFolder": "/workspace",
    "runArgs": [
        "--init",
        "--cap-add=SYS_PTRACE",
        "--security-opt",
        "seccom

Main execution block

In [27]:
if flag == False:
    repos = generate_github_repos(prompt)
    files = retrieve_files(repos)
else:
    repo = [prompt]
    files = retrieve_files(repo)

Successfully retrieved README.md from https://raw.githubusercontent.com/nodejs/node/main/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/nodejs/node/main/benchmark/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/nodejs/node/main/deps/acorn/acorn-walk/README.md
Successfully retrieved package.json from https://raw.githubusercontent.com/nodejs/node/main/deps/acorn/acorn-walk/package.json
Successfully retrieved README.md from https://raw.githubusercontent.com/nodejs/node/main/deps/acorn/acorn/README.md
Successfully retrieved package.json from https://raw.githubusercontent.com/nodejs/node/main/deps/acorn/acorn/package.json
Successfully retrieved README.md from https://raw.githubusercontent.com/nodejs/node/main/deps/ada/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/nodejs/node/main/deps/cares/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/nodejs/node/main